In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go # potrzebne do stworzenia ladnej tabelki
from dash import Dash, html, dcc, dash_table, no_update
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
import re

In [2]:
df = pd.read_csv("avocado.csv")
df = (df.drop(columns=["Unnamed: 0", "4046", "4225", "4770"])
      .assign(Date = pd.to_datetime(df['Date']),
              type = df['type'].astype('category'),
              region = df['region'].apply(lambda _x: re.sub(r"(\w)([A-Z])", r"\1 \2", _x)).astype('category')))

In [8]:
df.head()

,Date,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [30]:
# dane do tabeli w dashboardzie
long_df = pd.melt(df, 
id_vars=['Date', 'region', 'year', 'type'], 
value_vars=df.columns[df.columns.str.contains("^(?!Total).*Bags", regex=True)],
var_name='bag_size',
value_name='volume',
ignore_index=True)

In [74]:
# zdefiniowanie komorki, ktora ma sie podswietlac w tabeli przy odpalaniu dashboardu
initial_active_cell = {"row": 0, "column": 1, "column_id": "region", "row_id": 0}
PAGE_SIZE = 10

In [80]:
load_figure_template('JOURNAL')
app = Dash(__name__, external_stylesheets=[dbc.themes.JOURNAL])

### app layout - includes dash components (graphs, chechboxes, any html needed)
app.layout = html.Div(children=[
    # header, title of the web page
    html.H1(children="Avocados sold in US between 2015 and 2018", 
            style={'text-align': 'center' 
                #    'color': 'green'
                   }),
    html.Br(),
    html.Div(children=[
    html.Div(children=[
    html.Label("Please, select the year"),
    html.Br(),
    dcc.Dropdown(id="select_year",
                #  className="form-label mt-4",
                 options=[{"label": str(rok), "value": rok} for rok in df['year'].unique()],
    multi=False,
    value=2015,
    style={'width': "95%"}
    )], style={'width': '15%', 'float':'left', 'display': 'inline-block'}),
 
    # html.Div(id='output_container', children=[]),
    html.Div(children=[
    html.Label("Please, select the country"),
    html.Br(),
    dcc.Dropdown(id='select_region', 
                #  className="form-label mt-4",
                 options=[{"label": x, "value": x} for x in df.query("region != 'Total US'")['region'].unique()],
    multi=False,
    value='Los Angeles',
    style={'width': "40%"}
    )], style={'width': '85%', 'float': 'left', 'display': 'inline-block'})]),
  
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),
    html.Br(),

    # html.H3("Average price and total volume of different types of avocado", style={'text-align': 'left'}),
    # html.H3(id="subtitle_container", children=[], style={'text-align': 'left'}),
    # html.Br(),

    html.Div([
    html.Div([dcc.Graph(id='line_plot_diff_cntry_organic', figure={}),
              dcc.Graph(id='line_plot_diff_cntry_conv', figure={}),
              dash_table.DataTable(id='table_output',
                                   columns=[{"name": i, "id": i} for i in long_df.columns],
                                #    data=long_df,
                                   page_current=0,
                                   page_size=PAGE_SIZE,
                                   page_action='custom',
                                   active_cell=initial_active_cell
                                   )
              ], 
              style={'display': 'inline-block', 
                     'vertical-align':'top',
                     'width': '50%'}), 
    html.Div([dcc.Graph(id='total_bags_plot', figure={}),
              dcc.Graph(id="table-based-plot", figure={})],
              style={'display': 'inline-block',
                     'vertical-align':'bottom',
                     'width': '50%'})]),

    html.Div(className='gap'),
    html.Div(className='li'),
    html.Div(className='gap')
                       
])

### connecting the plotly graphs with dash components

@app.callback(
    [
    #  Output(component_id="subtitle_container", component_property='children'),
     Output(component_id='line_plot_diff_cntry_organic', component_property='figure'),
     Output(component_id='line_plot_diff_cntry_conv', component_property='figure'),
     Output(component_id='total_bags_plot', component_property='figure'),
     Output(component_id='table_output', component_property='data')], 
    [Input(component_id='select_year', component_property='value'),
    Input(component_id='select_region', component_property='value'),
    Input(component_id='table_output', component_property='page_current'),
    Input(component_id='table_output', component_property='page_size')]
)

# after each callback you have to put a function to define it, if you have one Input, then there is just one
# argument in the following function
def update_graph(option_selected, snd_option_selected, page_current, page_size):
    container = f"The year chosen by the user was: {option_selected}"
    # container2 = ' '.join(re.findall('.[^A-Z]*',snd_option_selected))

    dff = df.copy()

    # plotly express figure

    df_fig = dff.query("year == @option_selected and region == @snd_option_selected")
    df_fig = df_fig.assign(text = lambda x_: np.where(x_['Date'] == x_['Date'].max(), x_['type'], ""))

    fig = px.line(
        data_frame=df_fig,
        x='Date',
        y='AveragePrice',
        text='text',
        color='type',
        labels={
        'Date':'',
        'AveragePrice':'average price'}
    )
    # fig.update_layout(legend=dict(title_font_family="Times New Roman",
    #                           font=dict(
    #     family="Courier",
    #         size=12,
    #         color="black"
    #                           )))
    fig.update_layout(showlegend=False,
                      title=dict(text=f"Average price of avocado sold in {option_selected} in {snd_option_selected}", 
                                x=0.08
                                #  automargin=True, 
                                #  yref='container'
                                ),
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
    fig.update_traces(textposition="top center",
                      hovertemplate='Date: %{x} <br>Average avocado price: %{y}')
  

    fig2 = px.line(
        data_frame=df_fig,
        x='Date',
        y='Total Volume',
        color='type',
        text='text',
        labels={
        'Date':'',
        'Total Volume':'total volume'}

    )
    fig2.update_layout(showlegend=False,
                       title=dict(text=f"Total volume of Avocado sold in {option_selected} in {snd_option_selected}",
                                  x=0.08),
                       xaxis=dict(showgrid=False),
                       yaxis=dict(showgrid=False))
    fig2.update_traces(textposition="top center",
                       hovertemplate='Date: %{x} <br>Total vol of avocado sold: %{y}')

    fig3 = px.bar(
        data_frame=(dff.groupby(['type', 'year', 'region'], as_index=False)['Total Bags']
                    .sum()
                    .query("region == @snd_option_selected")
                    .reset_index(drop=True)
                    .assign(year = lambda x_: x_['year'].astype(str))
                    ),
        x='type',
        y='Total Bags',
        color='year',
        barmode='relative',
        labels={
        'type': ''},
        text_auto='.2s'
    )

    fig3.update_layout(title=dict(text=f"Total number bags of avocado sold in {option_selected} in {snd_option_selected}", x=0.11), 
                       yaxis=dict(showgrid=False))
        # , legend=dict(title_font_family="Times New Roman",
        #                       font=dict(family="Courier",
        #                                 size=12,
        #                                 color="black"
        #                       ))

    fig3.update_traces(hovertemplate='Year: %{color} <br>Total no of bags sold: %{y}')
    fig3.update_yaxes(visible=False)

    table_df = (dff.query("year == @option_selected")
                .groupby(['type'], as_index=False)
                .apply(lambda x: x.nlargest(5, 'Total Volume'))
                .reset_index(drop=True)
                .drop(columns=['year'])
                .assign(Date = lambda x_: x_['Date'].dt.date))
    df_col_names = [' '.join(re.findall('.[^A-Z]*',i_)) for i_ in table_df.columns]

    # fig4 = go.Figure(go.Table(header=dict(values=df_col_names,
    #                                       line_color='#f4f6f6', 
    #                                       fill_color='white',
    #                                       align='center'), 
    #                           cells=dict(values=table_df.transpose().values.tolist(), 
    #                                      align='center',
    #                                      line_color="#f4f6f6",
    #                                      fill_color='white')))

    fig4 = (long_df.query("year == @option_selected")
            .assign(Date = long_df['Date'].dt.date)
            .iloc[page_current*page_size:(page_current+ 1)*page_size]
            .to_dict('records'))

    return fig, fig2, fig3, fig4 # we have defined two outputs, therefore the function needs to return two outputs

# wykres na podstawie danych z tabelki
@app.callback(
    [Output("table-based-plot", "figure")],
    [Input("table_output", "active_cell"),
    Input('select_year', component_property='value')])

# tworzenie wykresu na podstawie danych podswietlonych w tabelce 
def cell_clicked(active_cell, option_selected):
    print(active_cell)
    if active_cell is None:
        return no_update

    df_plot = long_df.query("year == @option_selected").assign(Date = long_df['Date'].dt.date)

    row = active_cell["row"]
    # col = active_cell["column_id"]
    tbl_selected_region = df_plot.at[row, "region"] 
    type_selected = df_plot.at[row, "type"] 

    fig5 = px.line(data_frame=long_df.query("region == @tbl_selected_region and type==@type_selected").sort_values(['Date']),
                   x='Date',
                   y='volume',
                   color='bag_size',
                   labels={'Date':'', 'volume':'different sized bags volume sold'})
    
    return fig5
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: on
{'row': 0, 'column': 1, 'column_id': 'region', 'row_id': 0}
{'row': 9, 'column': 1, 'column_id': 'region'}
{'row': 3, 'column': 4, 'column_id': 'bag_size'}
{'row': 3, 'column': 4, 'column_id': 'bag_size'}
{'row': 2, 'column': 2, 'column_id': 'year'}
{'row': 1, 'column': 1, 'column_id': 'region'}
{'row': 0, 'column': 1, 'column_id': 'region'}
None
